In [1]:
import pandas as pd
import numpy as np
import nltk
import re
# import spacy 
# from spacytextblob.spacytextblob import SpacyTextBlob

In [332]:
nlp = spacy.load('en_core_web_trf')
nlp.add_pipe('spacytextblob')

In [ ]:
text = "Based on histologic and immunophenotypic evidence alone, the findings do not qualify for antibody-mediated rejection"

doc = nlp(text)


In [ ]:
doc._.polarity      # Polarity: -0.125


In [4]:
data = pd.read_excel("renal_reports.xlsx")

In [40]:
len(data["Raw Case Text"].tolist()[0].split())

543

In [6]:
c = 0
c2,c3=0,0
def get_diag_list(report,mode="main",col="none"): 
    diag,flag  = [],0
#     sent_text = nltk.sent_tokenize(report)
#     for sent in sent_text:
#         if "FINAL DIAGNOSIS" in sent and not flag:
#             diag.append(sent)
#             flag = 1
#         elif flag:
#             diag.append(sent)
#             if "MICROSCOPIC DESCRIPTION" in sent:
#                 break
    words = report.split()
    for word in words:
        if word == "DIAGNOSIS" or "<DIAGNOSIS" in word:
            flag = 1
        elif flag:
            if word == "MICROSCOPIC" or word == "Edited":
                break
            else:
                diag.append(word)
    text = " ".join(diag)
#     result = re.findall('\-\s(.*?)\.', text)
    result,result_other = [],[]
    for sent in text.split("."):
        if "rejection" in sent or "infection" in sent:
            result.append(sent)
        elif "interstitial fibrosis" in sent and "tubular atrophy" in sent:
            result_other.append(sent)
            
            
    ifta_result = "noinfo"
    if result_other: 
        global c2
        c2+=1
        for i in result_other:
            if "no significant" in i.lower():
                ifta_result = "nosig"
            elif "minimal" in i.lower():
                ifta_result= "minimal"
            elif "mild" in i.lower():
                ifta_result = "mild"
            elif "moderate" in i.lower():
                ifta_result = "moderate"
            elif "severe" in i.lower():
                ifta_result = "severe"
            else:
                pass
                #print(i,f"---{c2}---")

                
                
    tcmr,abmr,poly_infection,rejection = 0,0,0,0
    if result:
        global c3
        c3+=1
        for i in result:
            if (not "no evidence" in i.lower()) and ("rejection" in i.lower()) \
                and (not "insufficient" in i.lower()) and (("t-cell" in i.lower()) or ("t cell" in i.lower())) and (not "below" in i.lower())\
                and (not "no definitive evidence" in i.lower()) and (not "no evid1ence" in i.lower())\
                and (not "cannot" in i.lower()) and (not "negative " in i.lower())\
                and (not "unlikely" in i.lower()) and (not "against this possibility" in i.lower()):
                #print(i,f"---{c3}---")
                tcmr = 1
            elif (not "no evidence" in i.lower()) and ("rejection" in i.lower()) \
                and (not "insufficient" in i.lower()) and ("antibody" in i.lower()) and (not "not " in i.lower())\
                and (not "no definitive evidence" in i.lower()) and (not "no evid1ence" in i.lower())\
                and (not "negative " in i.lower()):
                abmr = 1
                #print(i,f"---{c3}---")
            elif "polyomavirus" in i and (not "no evidence" in i.lower()) and (not "negative" in i.lower())\
                and (not "no definitive evidence" in i.lower()) and (not "no evid1ence" in i.lower()):
                #print(i,f"---{c3}---")
                poly_infection = 1
            
    if (tcmr or abmr):
        rejection = 1
                
    if col == "tcmr":
        return tcmr
    elif col == "abmr":
        return abmr
    elif col == "rejection":
        return rejection
    elif col == "poly_infection":
        return poly_infection
    elif col == "ifta":
        return ifta_result
    return result if mode == "main" else result_other

In [7]:
result = re.findall('\-\s(.*?)\.', text)
#print(get_diag_list(data["Raw Case Text"][0]))

In [8]:
# data["diag_lst"] = 
data["TCMR"] = data["Raw Case Text"].apply(get_diag_list,col="tcmr")
data["ABMR"] = data["Raw Case Text"].apply(get_diag_list,col="abmr")
data["isRejection"] = data["Raw Case Text"].apply(get_diag_list,col="rejection")
data["poly_infection"] = data["Raw Case Text"].apply(get_diag_list,col="poly_infection")
data["IFTA"] = data["Raw Case Text"].apply(get_diag_list,col="ifta")

data["main_categories"] = data["Raw Case Text"].apply(get_diag_list)
data["other_categories"] = data["Raw Case Text"].apply(get_diag_list,mode="other")

In [9]:
data.groupby(['ABMR']).size()

ABMR
0    3306
1     123
dtype: int64

In [34]:
def have_target(sent):
    target_lst = ["t-cell-mediated rejection", "antibody-mediated rejection","polyomavirus",\
                 "interstitial fibrosis","tubular atrophy"]
    for target_word in target_lst:
        if target_word in sent.lower():
            return True
    return False

def get_train(report):
    sents = report.split(".")
    result = []
    flag = False
    for sent in sents:
        if "microscopic description" in sent.lower():
            result.append(sent)
            flag = True
            continue
#         elif "interpretation" in sent.lower():
#             break
        elif "banff scoring components" in sent.lower():
            break
            
        if flag and not have_target(sent):
            result.append(sent)
            
    return ". ".join(result)

In [2]:
data = pd.read_csv("data.csv")

In [25]:
tcmr=["insufficient evidence for acute T-cell-mediated rejection",\
   "No evidence of acute T-cell-mediated rejection",\
   "without obvious evidence of outright acute T-cell-mediated rejection",\
   "the findings of which are below the diagnostic threshold for acute T-cell-mediated rejection",\
   "The presence of diffuse interstitial inflammation (i3) and severe tubulitis (t3) support the diagnosis of type IB acute T-cell-mediated rejection",\
   "falls below the diagnostic threshold of outright acute T-cell-mediated rejection",\
   "The findings are most consistent with acute T-cell-mediated rejection",\
   "The biopsy shows evidence of type IA acute T-cell-mediated rejection",\
   "The biopsy shows both acute T-cell-mediated (type IB) and acute antibody-mediated (C4d-positive) rejection",\
   "borderline change is present without sufficient evidence for outright acute T-cell-mediated rejection",\
   "acute T-cell-mediated rejection (type IA) is present supported",\
   "acute T-cell-mediated rejection was present in the patient's last biopsy",\
   "the findings are only sufficient for borderline changes and not outright acute T-cell-mediated rejection",\
   "the presence of neutrophils favors an infectious process rather than borderline changes or acute T-cell-mediated rejection",\
   "which is consistent with borderline changes and well below the diagnostic threshold of acute T-cell-mediated rejection",\
   "These findings are worrisome for acute T-cell-mediated rejection",\
   "no evidence of acute T-cell-mediated rejection",\
   "arguing strongly against acute T-cell-mediated rejection",\
   "the findings atypical for acute T-cell-mediated rejection",\
   "There is no evidence of a T-cell-mediated component to the rejection",\
   "the relative severity of the tubulitis present in this case raises the possibility of a superimposed component of acute T-cell-mediated rejection",\
   "The biopsy shows changes of acute T-cell-mediated rejection",\
   "superimposed acute T-cell-mediated rejection cannot be absolutely excluded",\
   "Such changes are consistent with acute T-cell-mediated rejection",
   "The biopsy shows both evidence of C4d-positive acute/active antibody-mediated rejection (AMR) and acute T-cell-mediated rejection",\
   "which are consistent with acute T-cell-mediated rejection",\
   "These findings are consistent with type IIB acute T-cell-mediated rejection",\
   "acute T-cell-mediated rejection cannot be absolutely excluded",\
   "The biopsy shows acute T-cell-mediated rejection",\
   "the tubulointerstitial features are not sufficient to diagnose acute T-cell-mediated rejection",\
   "findings are only sufficient to qualify as borderline changes and not outright acute T-cell-mediated rejection",
   "Course has been complicated by persistent IB T-cell mediated rejection",\
   "The biopsy shows persistent acute T-cell-mediated rejection",\
   "acute T-cell-mediated rejection cannot be excluded",\
   "a single artery with endotheliitis is diagnostic of acute T-cell-mediated rejection",\
   "biopsy showed Banff grade IB T-cell mediated rejection",\
   "The biopsy demonstrates persistent acute T-cell-mediated rejection",\
   "The biopsy shows vascular rejection in the form of acute T-cell-mediated rejection",\
   "The biopsy shows changes of both acute T-cell-mediated rejection and acute antibody-mediated rejection",\
   "consistent with type IB acute T-cell-mediated rejection",\
   "course has been complicated by IB T-cell mediated rejection",\
   "The biopsy shows evidence of acute T-cell-mediated rejection",\
   "The biopsy demonstrates acute T-cell-mediated rejection",\
   "The findings are those of acute T-cell-mediated rejection",\
   "These findings are worrisome for acute T-cell-mediated rejection",\
   "Such findings can be classified as acute T-cell-mediated rejection",\
   "These findings are consistent with acute T-cell-mediated rejection",\
   "the biopsy demonstrates evidence that is sufficient for borderline changes (i1/t1) but not outright acute T-cell-mediated rejection",\
   "The biopsy shows well-developed changes of acute T-cell-mediated rejection",\
   "He had recent acute T-cell-mediated rejection, type IB, and acute/active antibody-mediated rejection",\
   "The biopsy shows persistent acute T-cell-mediated rejection",\
   "The biopsy shows evidence of type IA acute T-cell-mediated rejection",\
   "The biopsy shows many pathologic changes including type IIB acute T-cell-mediated rejection and C4d-positive antibody-mediated rejection",\
   "does not represent acute T-cell-mediated rejection",\
   "changes with i1 and t2 likely represents resolving acute T-cell-mediated rejection",\
   "There is no evidence of acute antibody-mediated rejection",\
   "rising creatinine revealed borderline changes suspicious for treated T-cell mediated acute rejection",\
   "which is consistent with acute T-cell-mediated rejection",\
   "isolated endotheliitis would qualify for the diagnosis of acute T-cell-mediated rejection",\
   "The biopsy shows evidence of vascular rejection in the form of type IIA acute T-cell-mediated rejection",\
   "acute T-cell-mediated rejection cannot be excluded",\
   "biopsy was suspicious for acute T-cell mediated rejection",\
   "consistent with acute T-cell-mediated rejection",\
   "There is no evidence of T-cell-mediated rejection",\
   "These findings are sufficient for the diagnosis of acute T-cell-mediated rejection",\
   "which shows evidence of acute T-cell-mediated rejection",\
   "The findings are those of acute T-cell-mediated rejection",\
   "the inflammation and tubulitis is highly suspicious for superimposed acute T-cell-mediated rejection",\
   "acute T-cell-mediated rejection cannot be absolutely excluded",\
   "no evidence of acute T-cell-mediated rejection",\
   "even case of isolated endotheliitis would qualify for the diagnosis of acute T-cell-mediated rejection",\
   "which raises the possibility of a component of superimposed acute T-cell-mediated rejection",\
   "whether T-cell-mediated rejection is truly present cannot be determined"
   
]

abmr=["No evidence of acute antibody-mediated rejection",\
   "which showed borderline changes and features of chronic antibody mediated rejection",\
   "No significant microvascular inflammation is present to suggest acute antibody-mediated rejection",\
   "These latter findings support the diagnosis of acute antibody-mediated rejection",\
   "These findings are consistent with acute antibody-mediated rejection",\
   "No significant microvascular inflammation is present to suggest antibody-mediated rejection",\
   "These findings should be correlated to the patient's donor specific antibody status to determine whether there is enough evidence to support C4d-negative chronic active antibody-mediated rejection",\
   "The biopsy shows findings that are consistent with C4d-positive acute/active antibody-mediated rejection",\
   "The biopsy shows both acute T-cell-mediated (type IB) and acute antibody-mediated (C4d-positive) rejection",\
   "Transplant glomerulopathy and transplant arteriopathy are present as evidence for the chronic component of antibody-mediated rejection",\
   "which raises the possibility of acute antibody-mediated rejection",\
   "Whether the microvascular inflammation seen in the current biopsy is related to antibody-mediated rejection is also unclear",\
   "these findings support acute/active antibody-mediated rejection",\
   "The presence of neutrophilic glomerulitis raises the possibility of acute antibody-mediated rejection",\
   "These findings suggest an active component to the antibody-mediated rejection",\
   "should be correlated to donor specific antibody status to determine whether the patient qualifies as having C4d-negative chronic active antibody-mediated rejection",\
   "chronic antibody-mediated rejection diagnosed on biopsy",\
   "interstitial neutrophils are likely residual inflammation from prior acute/active antibody-mediated rejection",\
   "correlation to donor specific antibodies is required to establish whether there is an active component of antibody-mediated rejection",\
   "No evidence of antibody-mediated rejection",\
   "The biopsy shows histologic evidence of acute/active antibody-mediated rejection",\
   "These findings need to be correlated to donor specific antibody status to exclude an active component of antibody-mediated rejection",\
   "The findings are those of acute/active antibody-mediated rejection",\
   "with evidence of chronic antibody mediated rejection",\
   "These features indicate a superimposed component of chronic active antibody-mediated rejection",\
   "donor specific antibody status is required to determine if the patient has an active component of antibody-mediated rejection",\
   "no evidence of acute antibody-mediated rejection",\
   "these findings should be correlated to donor specific antibody results to determine if there is an active component of antibody-mediated rejection",\
   "The biopsy shows both evidence of C4d-positive acute/active antibody-mediated rejection (AMR) and acute T-cell-mediated rejection",\
   "these findings must be correlated to the patient's donor specific antibody status to determine if there is a component of C4d-negative antibody-mediated rejection",\
   "These findings need to be correlated to donor specific antibody status in order to determine if there is enough evidence for chronic active antibody-mediated rejection",\
   "these findings are insufficient for the diagnosis of acute antibody-mediated rejection",\
   "finding is uncertain and should be correlated to the donor specific antibody status to exclude C4d-negative acute antibody-mediated rejection",\
   "due to an antibody-mediated rejection",\
   "With regard to antibody-mediated rejection",\
   "there is no significant microvascular inflammation to suggest acute antibody-mediated rejection",\
   "which is the below the diagnostic threshold of acute antibody-mediated rejection",\
   "these findings are suspicious for C4d-negative acute antibody-mediated rejection",\
   "These findings should be correlated to the donor specific antibody status to determine if there is sufficient evidence of acute/active antibody-mediated rejection",\
   "If donor specific antibodies are elevated, then these findings would be best classified as chronic active antibody-mediated rejection",\
   "No evidence of antibody-mediated rejection",\
   "findings are insufficient for acute antibody-mediated rejection",\
   "which is suspicious for acute/active antibody-mediated rejection",\
   "No features of acute antibody-mediated rejection",\
   "there is no evidence of acute antibody-mediated rejection",\
   "These findings should be correlated to the patient's donor specific antibody status to determine whether C4d-negative antibody-mediated rejection should be considered",\
   "if donor specific antibodies are elevated, then these features most likely represent acute/active antibody-mediated rejection",\
   "biopsy on 7/14/2017 showing evidence of antibody mediated rejection",\
   "no microvascular inflammation of other indication of significant tissue injury is present to suggest acute/active antibody-mediated rejection",\
   "should be correlated to the donor specific antibody status to exclude C4d-negative acute antibody-mediated rejection",\
   "these findings are still suspicious but not entirely diagnostic for acute antibody-mediated rejection",\
   "these findings should be correlated to current donor specific antibody levels to determine if there is an active component of antibody-mediated rejection",\
   "recent treatment for antibody-mediated rejection",\
   "The biopsy shows changes of both acute T-cell-mediated rejection and acute antibody-mediated rejection",\
   "these findings are consistent with acute-active antibody-mediated rejection",\
   "There is no evidence of acute antibody-mediated rejection",\
   "These findings should be correlated to the patient's donor specific antibody status to determine if there is a component of acute/active antibody-mediated rejection",\
   "the findings support persistent acute/active antibody-mediated rejection",\
   "the findings are insufficient for antibody-mediated rejection",\
   "the biopsy findings are insufficient to suggest acute antibody-mediated rejection",\
   "there is insufficient histologic and immunofluorescence evidence for acute antibody-mediated rejection",\
   "The lack of microvascular inflammation and C4d staining in peritubular capillaries argue against antibody-mediated rejection",\
   "These findings could be due to acute antibody-mediated rejection",\
   "acute antibody-mediated rejection (AMR) are the most frequently encountered causes of TMA",\
   "neutrophilic glomerulitis is likely related to ischemia-reperfusion injury rather than an antibody-mediated rejection",\
   "These findings are insufficient for acute antibody-mediated rejection",\
   "He had recent acute T-cell-mediated rejection, type IB, and acute/active antibody-mediated rejection",\
   "The C4d may be lingering from the recent prior episode of antibody-mediated rejection",\
   "No significant microvascular inflammation is seen to suggest acute antibody-mediated rejection",\
   "Correlation with donor specific antibody status is needed to determine whether acute antibody-mediated rejection is present",\
   "which argues against acute antibody-mediated rejection",\
   "The presence of moderate glomerulitis and mild peritubular capillaritis is concerning for acute antibody-mediated rejection",\
   "supporting an acute/active component of antibody-mediated rejection",\
   "These findings should be correlated to donor specific antibody status to determine whether antibody-mediated rejection is present",\
   "The biopsy shows many pathologic changes including type IIB acute T-cell-mediated rejection and C4d-positive antibody-mediated rejection",\
   "biopsy on 2/1/2016 showing no acute cellular or antibody-mediated rejection",\
   "mild peritubular capillaritis is concerning for acute antibody-mediated rejection",\
   "including antibody-mediated rejection and acute calcineurin inhibitor (CNI) nephrotoxicity",\
   "suggest concurrent antibody-mediated rejection",\
   "The biopsy shows features of chronic active antibody-mediated rejection",\
   "these findings require correlation to the patient's current donor-specific antibody status to determine whether there is enough evidence to qualify for C4d-negative acute antibody-mediated rejection",\
   "There is no evidence of acute antibody-mediated rejection",\
   "These findings should be correlated to donor-specific antibody status to determine whether there is presence of acute antibody-mediated rejection",\
   "the microvascular inflammation may indicate acute antibody-mediated rejection",\
   "whether there are other findings to suggest antibody-mediated rejection",\
   "the biopsy is highly suspicious for concurrent antibody-mediated rejection",\
   "whether this biopsy represents chronic active antibody-mediated rejection (that is C4d negative) cannot be determined",\
   "biopsy on 11/3/2014 showed antibody-mediated rejection",\
   "which are likely related to ischemia/reperfusion injury rather than antibody-mediated rejection",\
   "which is likely related to ischemia/reperfusion injury rather than an antibody-mediated rejection",\
   "mild peritubular capillaritis is concerning for antibody-mediated rejection",\
   "there is no evidence of antibody-mediated rejection",\
   "These findings are worrisome for antibody-mediated rejection",\
   "TMA in the transplant setting can be due to a variety of etiologies including calcineurin inhibitor nephrotoxicity, severe hypertension, and antibody-mediated rejection",\
   "He was recently treated with IVIG and Rituximab for antibody-mediated rejection",\
   "If the patient has elevated donor specific antibodies, then these findings would qualify for chronic active antibody-mediated rejection",\
   "are unlikely a result of antibody-mediated rejection",\
   "thus superimposed acute antibody-mediated rejection is unlikely",\
   "if donor specific antibodies are positive, then the findings qualify for antibody-mediated rejection",\
   "IHC stains were performed to exclude post-transplant lymphoproliferative disorder as well as polyomavirus infection and C4d-positive antibody-mediated rejection",\
   "are present to suggest chronic active antibody-mediated rejection",\
   "no evidence of significant microvascular inflammation or C4d positivity to support antibody-mediated rejection",\
   "This significance of this lesion may depend on time frame post-transplantation and whether there are other findings to suggest antibody-mediated rejection",\
   "no evidence to support antibody-mediated rejection",\
   "which is likely related to the surgical procedure rather than an antibody-mediated rejection",\
   "These features would be consistent with acute/active antibody-mediated rejection",\
   "the presence of substantial IF/TA would place this biopsy in the category of chronic active antibody-mediated rejection",\
   "the findings are consistent with acute/active antibody-mediated rejection",\
   "no morphologic or immunohistochemical evidence of antibody-mediated rejection"
]

In [24]:
for sent in data["train_report_qa"]:
    flag=False
    if str(sent)!= "nan":
        for i in sent.split("."):
            if  ((("t-cell" in i.lower()) or ("t cell" in i.lower())) or ("antibody" in i.lower())) and ("rejection" in i.lower()):
                flag = True
                print(i)
    print("SENTENCE END>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>\n") if flag else 0

   - Minimal interstitial inflammation with rare minimal tubulitis, insufficient evidence for acute T-cell-mediated rejection
   - No evidence of acute antibody-mediated rejection
SENTENCE END>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

   He underwent biopsy due to proteinuria on 7/2/2020, which showed borderline changes and features of chronic antibody mediated rejection with transplant glomerulopathy
SENTENCE END>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

   - No evidence of acute T-cell-mediated rejection
   - No evidence of acute antibody-mediated rejection
SENTENCE END>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

   - No evidence of acute T-cell-mediated rejection
   - No evidence of acute antibody-mediated rejection
SENTENCE END>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

   These thick sections show similar findings as those described i

In [4]:
for sent in data["train_report_qa"]:
    flag=False
    if str(sent)!= "nan":
        for i in sent.split("."):
            if  (not "no evidence" in i.lower()) and ("rejection" in i.lower()) \
                and (not "insufficient" in i.lower()) and (not "below" in i.lower())\
                and (not "no definitive evidence" in i.lower()) and (not "no evid1ence" in i.lower())\
                and (not "cannot" in i.lower()) and (not "negative " in i.lower())\
                and (not "unlikely" in i.lower()) and (not "against this possibility" in i.lower()) \
                and (not "no evidence" in i.lower()) and ("rejection" in i.lower()) \
                and (not "insufficient" in i.lower()) and (not "not " in i.lower())\
                and (not "no definitive evidence" in i.lower()) and (not "no evid1ence" in i.lower())\
                and (not "negative " in i.lower()):
                flag = True
                print(i)
    print("SENTENCE END>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>\n") if flag else 0

   He underwent biopsy due to proteinuria on 7/2/2020, which showed borderline changes and features of chronic antibody mediated rejection with transplant glomerulopathy
SENTENCE END>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

   These thick sections show similar findings as those described in the LM portion of this analysis, without obvious evidence of outright acute T-cell-mediated rejection (i
SENTENCE END>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

  
Borderline changes can be considered but the patient underwent recent treatment for
presumed acute rejection, which may have changed the initial histologic appearance
  
No significant microvascular inflammation is present to suggest acute
antibody-mediated rejection and C4d is negative
SENTENCE END>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

   There is no acute rejection but substantial
donor-derived renal pathology that is li

In [ ]:
["consistent with",\
 "the findings are those of",\
 "the evidence for infection is present",\
 "shows persistent",\
 "shows well-developed features",\
 "shows evidence",\
 "demonstrates evidence",\
 "shows prominent",\
 "there is evidence",\
 "the biopsy shows polyomavirus",\
 "this finding represents",\
 "is present",\
 "shows features",\
 "indicative of",\
 "no evidence", "negative", "no definitive evidence","no evid1ence"
]

In [13]:
for sent in data["train_report_qa"]:
    flag=False
    if str(sent)!= "nan":
        for i in sent.split("."):
            if  "polyomavirus" in i and (not "no evidence" in i.lower()) and (not "negative" in i.lower())\
                        and (not "no definitive evidence" in i.lower()) and (not "no evid1ence" in i.lower()):
                flag = True
                print(i)
    print("SENTENCE END>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>\n") if flag else 0

 
 
Interpretation:  The biopsy shows scattered tubules positive for SV40, consistent
with polyomavirus nephropathy
   Overall, the inflammatory load is minimal but
polyomavirus infection is often patchy and is difficult to ascertain the extent
based on a biopsy
SENTENCE END>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

   Thus, whether
the polyomavirus infection is contributing to any graft damage at this point is
unclear
SENTENCE END>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

 
 
Interpretation:  The findings are those of polyomavirus nephropathy
   Additionally, a focus of neutrophils is noted with
infiltration into the tubular epithelium, which is not typical of polyomavirus
infection
SENTENCE END>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

  
With that said, polyomavirus infection can be quite patchy and may not be sampled on
a small biopsy
SENTENCE END>>>>>>>>>>>>>>>>>>>>>>>

In [26]:
tcmr=["insufficient evidence for acute T-cell-mediated rejection",\
   "No evidence of acute T-cell-mediated rejection",\
   "without obvious evidence of outright acute T-cell-mediated rejection",\
   "the findings of which are below the diagnostic threshold for acute T-cell-mediated rejection",\
   "The presence of diffuse interstitial inflammation (i3) and severe tubulitis (t3) support the diagnosis of type IB acute T-cell-mediated rejection",\
   "falls below the diagnostic threshold of outright acute T-cell-mediated rejection",\
   "The findings are most consistent with acute T-cell-mediated rejection",\
   "The biopsy shows evidence of type IA acute T-cell-mediated rejection",\
   "The biopsy shows both acute T-cell-mediated (type IB) and acute antibody-mediated (C4d-positive) rejection",\
   "borderline change is present without sufficient evidence for outright acute T-cell-mediated rejection",\
   "acute T-cell-mediated rejection (type IA) is present supported",\
   "acute T-cell-mediated rejection was present in the patient's last biopsy",\
   "the findings are only sufficient for borderline changes and not outright acute T-cell-mediated rejection",\
   "the presence of neutrophils favors an infectious process rather than borderline changes or acute T-cell-mediated rejection",\
   "which is consistent with borderline changes and well below the diagnostic threshold of acute T-cell-mediated rejection",\
   "These findings are worrisome for acute T-cell-mediated rejection",\
   "no evidence of acute T-cell-mediated rejection",\
   "arguing strongly against acute T-cell-mediated rejection",\
   "the findings atypical for acute T-cell-mediated rejection",\
   "There is no evidence of a T-cell-mediated component to the rejection",\
   "the relative severity of the tubulitis present in this case raises the possibility of a superimposed component of acute T-cell-mediated rejection",\
   "The biopsy shows changes of acute T-cell-mediated rejection",\
   "superimposed acute T-cell-mediated rejection cannot be absolutely excluded",\
   "Such changes are consistent with acute T-cell-mediated rejection",
   "The biopsy shows both evidence of C4d-positive acute/active antibody-mediated rejection (AMR) and acute T-cell-mediated rejection",\
   "which are consistent with acute T-cell-mediated rejection",\
   "These findings are consistent with type IIB acute T-cell-mediated rejection",\
   "acute T-cell-mediated rejection cannot be absolutely excluded",\
   "The biopsy shows acute T-cell-mediated rejection",\
   "the tubulointerstitial features are not sufficient to diagnose acute T-cell-mediated rejection",\
   "findings are only sufficient to qualify as borderline changes and not outright acute T-cell-mediated rejection",
   "Course has been complicated by persistent IB T-cell mediated rejection",\
   "The biopsy shows persistent acute T-cell-mediated rejection",\
   "acute T-cell-mediated rejection cannot be excluded",\
   "a single artery with endotheliitis is diagnostic of acute T-cell-mediated rejection",\
   "biopsy showed Banff grade IB T-cell mediated rejection",\
   "The biopsy demonstrates persistent acute T-cell-mediated rejection",\
   "The biopsy shows vascular rejection in the form of acute T-cell-mediated rejection",\
   "The biopsy shows changes of both acute T-cell-mediated rejection and acute antibody-mediated rejection",\
   "consistent with type IB acute T-cell-mediated rejection",\
   "course has been complicated by IB T-cell mediated rejection",\
   "The biopsy shows evidence of acute T-cell-mediated rejection",\
   "The biopsy demonstrates acute T-cell-mediated rejection",\
   "The findings are those of acute T-cell-mediated rejection",\
   "These findings are worrisome for acute T-cell-mediated rejection",\
   "Such findings can be classified as acute T-cell-mediated rejection",\
   "These findings are consistent with acute T-cell-mediated rejection",\
   "the biopsy demonstrates evidence that is sufficient for borderline changes (i1/t1) but not outright acute T-cell-mediated rejection",\
   "The biopsy shows well-developed changes of acute T-cell-mediated rejection",\
   "He had recent acute T-cell-mediated rejection, type IB, and acute/active antibody-mediated rejection",\
   "The biopsy shows persistent acute T-cell-mediated rejection",\
   "The biopsy shows evidence of type IA acute T-cell-mediated rejection",\
   "The biopsy shows many pathologic changes including type IIB acute T-cell-mediated rejection and C4d-positive antibody-mediated rejection",\
   "does not represent acute T-cell-mediated rejection",\
   "changes with i1 and t2 likely represents resolving acute T-cell-mediated rejection",\
   "There is no evidence of acute antibody-mediated rejection",\
   "rising creatinine revealed borderline changes suspicious for treated T-cell mediated acute rejection",\
   "which is consistent with acute T-cell-mediated rejection",\
   "isolated endotheliitis would qualify for the diagnosis of acute T-cell-mediated rejection",\
   "The biopsy shows evidence of vascular rejection in the form of type IIA acute T-cell-mediated rejection",\
   "acute T-cell-mediated rejection cannot be excluded",\
   "biopsy was suspicious for acute T-cell mediated rejection",\
   "consistent with acute T-cell-mediated rejection",\
   "There is no evidence of T-cell-mediated rejection",\
   "These findings are sufficient for the diagnosis of acute T-cell-mediated rejection",\
   "which shows evidence of acute T-cell-mediated rejection",\
   "The findings are those of acute T-cell-mediated rejection",\
   "the inflammation and tubulitis is highly suspicious for superimposed acute T-cell-mediated rejection",\
   "acute T-cell-mediated rejection cannot be absolutely excluded",\
   "no evidence of acute T-cell-mediated rejection",\
   "even case of isolated endotheliitis would qualify for the diagnosis of acute T-cell-mediated rejection",\
   "which raises the possibility of a component of superimposed acute T-cell-mediated rejection",\
   "whether T-cell-mediated rejection is truly present cannot be determined"
   
]

abmr=["No evidence of acute antibody-mediated rejection",\
   "which showed borderline changes and features of chronic antibody mediated rejection",\
   "No significant microvascular inflammation is present to suggest acute antibody-mediated rejection",\
   "These latter findings support the diagnosis of acute antibody-mediated rejection",\
   "These findings are consistent with acute antibody-mediated rejection",\
   "No significant microvascular inflammation is present to suggest antibody-mediated rejection",\
   "These findings should be correlated to the patient's donor specific antibody status to determine whether there is enough evidence to support C4d-negative chronic active antibody-mediated rejection",\
   "The biopsy shows findings that are consistent with C4d-positive acute/active antibody-mediated rejection",\
   "The biopsy shows both acute T-cell-mediated (type IB) and acute antibody-mediated (C4d-positive) rejection",\
   "Transplant glomerulopathy and transplant arteriopathy are present as evidence for the chronic component of antibody-mediated rejection",\
   "which raises the possibility of acute antibody-mediated rejection",\
   "Whether the microvascular inflammation seen in the current biopsy is related to antibody-mediated rejection is also unclear",\
   "these findings support acute/active antibody-mediated rejection",\
   "The presence of neutrophilic glomerulitis raises the possibility of acute antibody-mediated rejection",\
   "These findings suggest an active component to the antibody-mediated rejection",\
   "should be correlated to donor specific antibody status to determine whether the patient qualifies as having C4d-negative chronic active antibody-mediated rejection",\
   "chronic antibody-mediated rejection diagnosed on biopsy",\
   "interstitial neutrophils are likely residual inflammation from prior acute/active antibody-mediated rejection",\
   "correlation to donor specific antibodies is required to establish whether there is an active component of antibody-mediated rejection",\
   "No evidence of antibody-mediated rejection",\
   "The biopsy shows histologic evidence of acute/active antibody-mediated rejection",\
   "These findings need to be correlated to donor specific antibody status to exclude an active component of antibody-mediated rejection",\
   "The findings are those of acute/active antibody-mediated rejection",\
   "with evidence of chronic antibody mediated rejection",\
   "These features indicate a superimposed component of chronic active antibody-mediated rejection",\
   "donor specific antibody status is required to determine if the patient has an active component of antibody-mediated rejection",\
   "no evidence of acute antibody-mediated rejection",\
   "these findings should be correlated to donor specific antibody results to determine if there is an active component of antibody-mediated rejection",\
   "The biopsy shows both evidence of C4d-positive acute/active antibody-mediated rejection (AMR) and acute T-cell-mediated rejection",\
   "these findings must be correlated to the patient's donor specific antibody status to determine if there is a component of C4d-negative antibody-mediated rejection",\
   "These findings need to be correlated to donor specific antibody status in order to determine if there is enough evidence for chronic active antibody-mediated rejection",\
   "these findings are insufficient for the diagnosis of acute antibody-mediated rejection",\
   "finding is uncertain and should be correlated to the donor specific antibody status to exclude C4d-negative acute antibody-mediated rejection",\
   "due to an antibody-mediated rejection",\
   "With regard to antibody-mediated rejection",\
   "there is no significant microvascular inflammation to suggest acute antibody-mediated rejection",\
   "which is the below the diagnostic threshold of acute antibody-mediated rejection",\
   "these findings are suspicious for C4d-negative acute antibody-mediated rejection",\
   "These findings should be correlated to the donor specific antibody status to determine if there is sufficient evidence of acute/active antibody-mediated rejection",\
   "If donor specific antibodies are elevated, then these findings would be best classified as chronic active antibody-mediated rejection",\
   "No evidence of antibody-mediated rejection",\
   "findings are insufficient for acute antibody-mediated rejection",\
   "which is suspicious for acute/active antibody-mediated rejection",\
   "No features of acute antibody-mediated rejection",\
   "there is no evidence of acute antibody-mediated rejection",\
   "These findings should be correlated to the patient's donor specific antibody status to determine whether C4d-negative antibody-mediated rejection should be considered",\
   "if donor specific antibodies are elevated, then these features most likely represent acute/active antibody-mediated rejection",\
   "biopsy on 7/14/2017 showing evidence of antibody mediated rejection",\
   "no microvascular inflammation of other indication of significant tissue injury is present to suggest acute/active antibody-mediated rejection",\
   "should be correlated to the donor specific antibody status to exclude C4d-negative acute antibody-mediated rejection",\
   "these findings are still suspicious but not entirely diagnostic for acute antibody-mediated rejection",\
   "these findings should be correlated to current donor specific antibody levels to determine if there is an active component of antibody-mediated rejection",\
   "recent treatment for antibody-mediated rejection",\
   "The biopsy shows changes of both acute T-cell-mediated rejection and acute antibody-mediated rejection",\
   "these findings are consistent with acute-active antibody-mediated rejection",\
   "There is no evidence of acute antibody-mediated rejection",\
   "These findings should be correlated to the patient's donor specific antibody status to determine if there is a component of acute/active antibody-mediated rejection",\
   "the findings support persistent acute/active antibody-mediated rejection",\
   "the findings are insufficient for antibody-mediated rejection",\
   "the biopsy findings are insufficient to suggest acute antibody-mediated rejection",\
   "there is insufficient histologic and immunofluorescence evidence for acute antibody-mediated rejection",\
   "The lack of microvascular inflammation and C4d staining in peritubular capillaries argue against antibody-mediated rejection",\
   "These findings could be due to acute antibody-mediated rejection",\
   "acute antibody-mediated rejection (AMR) are the most frequently encountered causes of TMA",\
   "neutrophilic glomerulitis is likely related to ischemia-reperfusion injury rather than an antibody-mediated rejection",\
   "These findings are insufficient for acute antibody-mediated rejection",\
   "He had recent acute T-cell-mediated rejection, type IB, and acute/active antibody-mediated rejection",\
   "The C4d may be lingering from the recent prior episode of antibody-mediated rejection",\
   "No significant microvascular inflammation is seen to suggest acute antibody-mediated rejection",\
   "Correlation with donor specific antibody status is needed to determine whether acute antibody-mediated rejection is present",\
   "which argues against acute antibody-mediated rejection",\
   "The presence of moderate glomerulitis and mild peritubular capillaritis is concerning for acute antibody-mediated rejection",\
   "supporting an acute/active component of antibody-mediated rejection",\
   "These findings should be correlated to donor specific antibody status to determine whether antibody-mediated rejection is present",\
   "The biopsy shows many pathologic changes including type IIB acute T-cell-mediated rejection and C4d-positive antibody-mediated rejection",\
   "biopsy on 2/1/2016 showing no acute cellular or antibody-mediated rejection",\
   "mild peritubular capillaritis is concerning for acute antibody-mediated rejection",\
   "including antibody-mediated rejection and acute calcineurin inhibitor (CNI) nephrotoxicity",\
   "suggest concurrent antibody-mediated rejection",\
   "The biopsy shows features of chronic active antibody-mediated rejection",\
   "these findings require correlation to the patient's current donor-specific antibody status to determine whether there is enough evidence to qualify for C4d-negative acute antibody-mediated rejection",\
   "There is no evidence of acute antibody-mediated rejection",\
   "These findings should be correlated to donor-specific antibody status to determine whether there is presence of acute antibody-mediated rejection",\
   "the microvascular inflammation may indicate acute antibody-mediated rejection",\
   "whether there are other findings to suggest antibody-mediated rejection",\
   "the biopsy is highly suspicious for concurrent antibody-mediated rejection",\
   "whether this biopsy represents chronic active antibody-mediated rejection (that is C4d negative) cannot be determined",\
   "biopsy on 11/3/2014 showed antibody-mediated rejection",\
   "which are likely related to ischemia/reperfusion injury rather than antibody-mediated rejection",\
   "which is likely related to ischemia/reperfusion injury rather than an antibody-mediated rejection",\
   "mild peritubular capillaritis is concerning for antibody-mediated rejection",\
   "there is no evidence of antibody-mediated rejection",\
   "These findings are worrisome for antibody-mediated rejection",\
   "TMA in the transplant setting can be due to a variety of etiologies including calcineurin inhibitor nephrotoxicity, severe hypertension, and antibody-mediated rejection",\
   "He was recently treated with IVIG and Rituximab for antibody-mediated rejection",\
   "If the patient has elevated donor specific antibodies, then these findings would qualify for chronic active antibody-mediated rejection",\
   "are unlikely a result of antibody-mediated rejection",\
   "thus superimposed acute antibody-mediated rejection is unlikely",\
   "if donor specific antibodies are positive, then the findings qualify for antibody-mediated rejection",\
   "IHC stains were performed to exclude post-transplant lymphoproliferative disorder as well as polyomavirus infection and C4d-positive antibody-mediated rejection",\
   "are present to suggest chronic active antibody-mediated rejection",\
   "no evidence of significant microvascular inflammation or C4d positivity to support antibody-mediated rejection",\
   "This significance of this lesion may depend on time frame post-transplantation and whether there are other findings to suggest antibody-mediated rejection",\
   "no evidence to support antibody-mediated rejection",\
   "which is likely related to the surgical procedure rather than an antibody-mediated rejection",\
   "These features would be consistent with acute/active antibody-mediated rejection",\
   "the presence of substantial IF/TA would place this biopsy in the category of chronic active antibody-mediated rejection",\
   "the findings are consistent with acute/active antibody-mediated rejection",\
   "no morphologic or immunohistochemical evidence of antibody-mediated rejection"
]

In [57]:
len(abmr+tcmr)

182

In [61]:
len(abmr)

109

In [42]:
s = """   But given the nonspecific nature of tubulitis in the setting of
infection, whether T-cell-mediated rejection is truly present cannot be determined
based on morphologic grounds alone"""
have_target_answer_tcmr(s)

   but given the nonspecific nature of tubulitis in the setting of infection, whether t-cell-mediated rejection is truly present cannot be determined based on morphologic grounds alone whether T-cell-mediated rejection is truly present cannot be determined 



(True, (78, 71))

In [60]:
import difflib

def get_overlap(s1, s2):
    s = difflib.SequenceMatcher(None, s1, s2)
    pos_a, pos_b, size = s.find_longest_match(0, len(s1), 0, len(s2)) 
    return s1[pos_a:pos_a+size],len(size)

get_overlap("whether T-cell-mediated rejection is truly present cannot be determined","rejection is truly present cannot be determined based on morphologic grounds alon")

'rejection is truly present cannot be determined'

In [47]:
def get_train_qa(report):
    sents = report.split(".")
    result = []
    flag = False
    pos_start,pos_end = 0,0
    for sent in sents:
        if "microscopic description" in sent.lower():
            result.append(sent)
            flag = True
            continue
#         elif "interpretation" in sent.lower():
#             break
        elif "banff scoring components" in sent.lower():
            break
            
        if flag:
            result.append(sent)
            
            
    return ". ".join(result)

def have_target_ifta(sent):
    target_lst = ["interstitial fibrosis","tubular atrophy"]
    for target_word in target_lst:
        if target_word in sent.lower():
            return True
    return False

def have_target_answer_ifta(sent):
    target_lst = ["severe", "moderate","mild","minimal","no significant"]
    for target_word in target_lst:
        if target_word in sent.lower():
            return True,(sent.lower().find(target_word),len(target_word))
    return False,-1

def get_train_qa_ifta(report):
    sents = report.split(".")
    pos_start,pos_end = 0,0
    for sent in sents:
        if have_target_ifta(sent):
            bool_tar, pos_info = have_target_answer_ifta(sent)
            if bool_tar:
                pos_start+=pos_info[0]
                pos_end = pos_start+pos_info[1]
                return (pos_start,pos_end)
        pos_start+=len(sent)+1
            
    return (0,0)     



def have_target_tcmr(sent):
    target_lst = tcmr
    for target_word in target_lst:
        if target_word.lower() in sent.lower().replace('\n', ' '):
            return True
    return False

def have_target_answer_tcmr(sent):
    target_lst = tcmr
    for target_word in target_lst:
        if target_word.lower() in sent.lower().replace('\n', ' '):
#             print(sent.lower().replace('\n', ' '),target_word,"\n")
            return True,(sent.lower().replace('\n', ' ').find(target_word.lower()),len(target_word))
    return False,-1

def get_train_qa_tcmr(report):
    sents = report.split(".")
    pos_start,pos_end = 0,0
    for sent in sents:
        if have_target_tcmr(sent):
            bool_tar, pos_info = have_target_answer_tcmr(sent)
            if bool_tar:
                pos_start+=pos_info[0]
                pos_end = pos_start+pos_info[1]
                return (pos_start,pos_end)
        pos_start+=len(sent)+1
            
    return (0,0)

def have_target_abmr(sent):
    target_lst = abmr
    for target_word in target_lst:
        if target_word.lower() in sent.lower().replace('\n', ' '):
            return True
    return False

def have_target_answer_abmr(sent):
    target_lst = abmr
    for target_word in target_lst:
        if target_word.lower() in sent.lower().replace('\n', ' '):
#             print(sent.lower().replace('\n', ' '),target_word,"\n")
            return True,(sent.lower().replace('\n', ' ').find(target_word.lower()),len(target_word))
    return False,-1

def get_train_qa_abmr(report):
    sents = report.split(".")
    pos_start,pos_end = 0,0
    for sent in sents:
        if have_target_abmr(sent):
            bool_tar, pos_info = have_target_answer_abmr(sent)
            if bool_tar:
                pos_start+=pos_info[0]
                pos_end = pos_start+pos_info[1]
                return (pos_start,pos_end)
        pos_start+=len(sent)+1
            
    return (0,0)


def have_target_poly(sent):
    target_lst = ["polyomavirus"]
    for target_word in target_lst:
        if target_word in sent.lower():
            return True
    return False

def have_target_answer_poly(sent):
    target_lst = ["consistent with",\
         "the findings are those of",\
         "the evidence for infection is present",\
         "shows persistent",\
         "shows well-developed features",\
         "shows evidence",\
         "demonstrates evidence",\
         "shows prominent",\
         "there is evidence",\
         "the biopsy shows polyomavirus",\
         "this finding represents",\
         "is present",\
         "shows features",\
         "indicative of",\
         "no evidence", "negative", "no definitive evidence","no evid1ence"]
    for target_word in target_lst:
        if target_word in sent.lower():
            return True,(sent.lower().find(target_word),len(target_word))
    return False,-1

def get_train_qa_poly(report):
    sents = report.split(".")
    pos_start,pos_end = 0,0
    for sent in sents:
        if have_target_poly(sent):
            bool_tar, pos_info = have_target_answer_poly(sent)
            if bool_tar:
                pos_start+=pos_info[0]
                pos_end = pos_start+pos_info[1]
                return (pos_start,pos_end)
        pos_start+=len(sent)+1
            
    return (0,0)     

In [28]:
data["train_report_qa"] = data["Raw Case Text"].apply(get_train_qa)

In [49]:
data["ifta_pos_qa"] = data["train_report_qa"].apply(get_train_qa_ifta)

In [45]:
data["tcmr_pos_qa"] = data["train_report_qa"].apply(get_train_qa_tcmr)

In [48]:
data["abmr_pos_qa"] = data["train_report_qa"].apply(get_train_qa_abmr)

In [18]:
data["ispoly_pos_qa"] = data["train_report_qa"].apply(get_train_qa_poly)

In [52]:
data["tcmr_class"] = data["tcmr_pos_qa"].apply(lambda x: 1 if x!=(0,0) else 0)

In [55]:
data["abmr_class"] = data["abmr_pos_qa"].apply(lambda x: 1 if x!=(0,0) else 0)

In [50]:
for report,pos in zip(data["train_report_qa"],data["abmr_pos_qa"]):
#     pos = eval(pos)
    if str(report) != "nan":
        print(report[pos[0]:pos[1]])





























No evidence of acute antibody-mediated rejection

















































































































































































































which showed borderline changes and features of chronic antibody mediated rejection
























































































































































































































































































































































































































































































































No evidence of acute antibody-mediated rejection
























































































which are likely related to ischemia/reperfusion injury
rather than antibody-mediated rejection













which is
likely related to ischemia/reperfusion injury rather than an antibody-mediated
rejection
which is
likely related to ischemia/reperfusion injury rather than an antibody-mediated
rejection












mild peritubular
capillaritis is concerning for antibody-mediated rejection




no evidence of antibody-mediated rejection














These findings are worrisome for antibody-mediated rejection

TMA in the transplant setting can be
due to a variety of etiologies including calcineurin inhibitor nephrotoxicity,
severe hypertension, and antibody-mediated rejection
no evidence of antibody-mediated rejection










He was recently treated with IVIG and Rituximab for antibody-mediated
rejection





If the patient has elevated donor specific antibodies, then these
findings would qualify for chronic active antibody-mediated rejection







are
unlikely a result of 

In [21]:
data["ispoly_pos_qa"][0]

(1181, 1189)

In [504]:
from os import listdir
from os.path import isfile, join

path_tcmr = "./Reports/TCMR"
onlyfiles = [f for f in listdir(path_tcmr) if isfile(join(path_tcmr, f))]

In [516]:
add_zero = lambda x: "0"*(5-len(x))+x
add_zeros = lambda x: x.split("-")[0]+"-"+add_zero(x.split("-")[-1])

names_tcmr = [add_zeros("".join(x[:-5].split())) for x in onlyfiles if "docx" in x]
names_tcmr_alts = [x[:2]+":SP"+add_zero(x.split("-")[-1]) for x in names_tcmr]

In [527]:
tcmr_notfounded = []

for n1,n2 in zip(names_tcmr,names_tcmr_alts):
    tcmr1 = data.loc[data['Accession #'] == n1, ['TCMR']]
    tcmr2 = data.loc[data['Accession #'] == n2, ['TCMR']]
    if not tcmr1.empty:
        print(n1,tcmr1,"\n")
    elif not tcmr2.empty:
        print(n2,tcmr2,"\n")
    else:
        tcmr_notfounded.append(n1)
        print(n1, "not found\n")



18SP-03373 not found

19SP-00552       TCMR
1829     1 

20SP-02432       TCMR
1198     1 

16:SP19332       TCMR
3128     1 

19SP-07918       TCMR
1622     1 

20SP-05151       TCMR
1110     1 

16:SP18352       TCMR
3161     1 

20SP-06383       TCMR
1065     1 

17:SP13115       TCMR
2892     1 

18SP-02741 not found

19SP-03472       TCMR
1740     1 

17:SP14330       TCMR
2849     1 

18SP-09429 not found

17SP-08896 not found

20SP-17376      TCMR
686     1 

18SP-03399 not found

17SP-03130 not found

18SP-17392       TCMR
1986     1 

17:SP15245       TCMR
2819     1 

19SP-12004       TCMR
1515     1 

18SP-21453       TCMR
1860     1 

17:SP22839       TCMR
2540     1 

16:SP14414       TCMR
3327     1 

20SP-10030      TCMR
938     1 

19SP-07293       TCMR
1637     1 

18SP-09287 not found

19SP-03213       TCMR
1744     1 

17SP-00307 not found

17SP-08687 not found

19SP-16400       TCMR
1419     1 

19SP-15522       TCMR
1441     1 

16:SP21868       TCMR
3035     1 

1

In [445]:
path_abmr = "./Reports/ABMR"
onlyfiles = [f for f in listdir(path_abmr) if isfile(join(path_abmr, f))]
names_abmr = [add_zeros("".join(x[:-5].split())) for x in onlyfiles if "docx" in x]
names_abmr_alts = [x[:2]+":SP"+add_zero(x.split("-")[-1]) for x in names_abmr]

In [528]:
abmr_notfounded=[]

for n1,n2 in zip(names_abmr,names_abmr_alts):
    abmr1 = data.loc[data['Accession #'] == n1, ['ABMR']]
    abmr2 = data.loc[data['Accession #'] == n2, ['ABMR']]
    if not abmr1.empty:
        print(n1,abmr1,"\n")
    elif not abmr2.empty:
        print(n2,abmr2,"\n")
    else:
        abmr_notfounded.append(n1)
        print(n1, "not found\n")



19SP-552 not found

20SP-00109       ABMR
1260     1 

20SP-17463      ABMR
682     1 

19SP-18229       ABMR
1369     1 

18:SP1887       ABMR
2215     1 

20SP-06383       ABMR
1065     1 

20SP-13355      ABMR
823     1 

19SP-18228       ABMR
1370     1 

19SP-17755       ABMR
1381     1 

20SP-03650       ABMR
1161     1 

19SP-02177       ABMR
1780     1 

19SP-09425       ABMR
1581     1 

19SP-03213       ABMR
1744     1 

19SP-09171       ABMR
1584     1 

18SP-16196       ABMR
2021     1 

18SP-04410 not found

19SP-02175       ABMR
1782     1 

18SP-11125       ABMR
2171     1 

18:SP197       ABMR
2214     1 

20SP-05437       ABMR
1101     1 

18SP-14833       ABMR
2060     1 

19SP-05042       ABMR
1698     1 

20SP-02432       ABMR
1198     1 

19SP-09165       ABMR
1586     1 

20SP-19264      ABMR
640     1
641     1 

18SP-13791       ABMR
2090     1 

20SP-10127      ABMR
932     1 

20SP-04575       ABMR
1133     1 

20SP-16047      ABMR
723     1 

19SP-17936      

In [529]:
tcmr_notfounded

['18SP-03373',
 '18SP-02741',
 '18SP-09429',
 '17SP-08896',
 '18SP-03399',
 '17SP-03130',
 '18SP-09287',
 '17SP-00307',
 '17SP-08687',
 '18SP-04447',
 '18SP-05585',
 '18SP-00197',
 '17SP-09713',
 '18SP-06186',
 '17SP-02781',
 '17SP-04507',
 '17SP-01586',
 '17SP-09948']

In [530]:
abmr_notfounded

['19SP-552', '18SP-04410']

In [531]:
names_abmr

['19SP-552',
 '20SP-00109',
 '20SP-17463',
 '19SP-18229',
 '18SP-1887',
 '20SP-06383',
 '20SP-13355',
 '19SP-18228',
 '19SP-17755',
 '20SP-03650',
 '19SP-02177',
 '19SP-09425',
 '19SP-03213',
 '19SP-09171',
 '18SP-16196',
 '18SP-04410',
 '19SP-02175',
 '18SP-11125',
 '18SP-197',
 '20SP-05437',
 '18SP-14833',
 '19SP-05042',
 '20SP-02432',
 '19SP-09165',
 '20SP-19264',
 '18SP-13791',
 '20SP-10127',
 '20SP-04575',
 '20SP-16047',
 '19SP-17936',
 '19SP-10973',
 '19SP-20644',
 '19SP-09600',
 '20SP-10669',
 '20SP-15631']

In [2]:
data = pd.read_csv("data.csv")

In [3]:
data

,Unnamed: 0,Accession #,Gender,Age at Procedure,Raw Case Text,TCMR,ABMR,isRejection,poly_infection,IFTA,main_categories,other_categories,train_report
0,0,21SP-22339,Male,47,"FINAL DIAGNOSIS \n TRANSPLANT KIDNEY, SURVEILL...",0,0,0,0,mild,[' - Minimal nonspecific interstitial inflamma...,[' - Mild interstitial fibrosis and tubular at...,", MICROSCOPIC DESCRIPTION \n Light Microscopy ..."
1,1,21SP-22334,Male,38,"FINAL DIAGNOSIS \n TRANSPLANT KIDNEY, SURVEILL...",0,0,0,0,minimal,[' - No evidence of acute T-cell-mediated reje...,[' - Minimal interstitial fibrosis and tubular...,", MICROSCOPIC DESCRIPTION \n Light Microscopy ..."
2,2,21SP-22330,Female,43,"FINAL DIAGNOSIS \n TRANSPLANT KIDNEY, SURVEILL...",0,0,0,0,nosig,"['\\n TRANSPLANT KIDNEY, SURVEILLANCE BIOPSY (...",[' - No significant interstitial fibrosis or t...,", MICROSCOPIC DESCRIPTION \n Light Microscopy ..."
3,3,21SP-22328,Male,46,"FINAL DIAGNOSIS \n TRANSPLANT KIDNEY, BIOPSY (...",0,0,0,0,nosig,[' - Insufficient evidence for acute T-cell-me...,[' - No significant interstitial fibrosis or t...,", MICROSCOPIC DESCRIPTION \n Light Microscopy ..."
4,4,21SP-22116,Male,27,"FINAL DIAGNOSIS \n TRANSPLANT KIDNEY, TIME-ZER...",0,0,0,0,nosig,[],"[' - No significant glomerulosclerosis, inters...",", MICROSCOPIC DESCRIPTION \n Light Microscopy ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3424,3424,16:SP11594,F,39,<Ordered Procedures \n G&M Level 4\nIMMUNOHIST...,0,0,0,0,mild,[' No evidence of T-cell-mediated or antibody-...,[' Mild interstitial fibrosis and tubular atro...,\njw:ds \nEdited By: 07/11/16 - 1516 16192><MI...
3425,3425,16:SP11519,M,55,<Ordered Procedures \n G&M Level 4\nIMMUNOHIST...,0,0,0,0,noinfo,[' No evidence of T-cell-mediated or antibody-...,[],\nEdited By: 07/08/16 - 1601 16192><MICROSCOPI...
3426,3426,16:SP11518,M,61,<Ordered Procedures \n G&M Level 4\nIMMUNOHIST...,0,0,0,0,moderate,[' No evidence of polyomavirus infection'],[' Moderate (approaching severe) interstitial ...,\nEdited By: 07/08/16 - 1701 16192><MICROSCOPI...
3427,3427,16:SP11507,F,29,<Ordered Procedures \n G&M Level 4\nIMMUNOHIST...,0,0,0,0,mild,[' No evidence of antibody-mediated rejection'...,[' Mild interstitial fibrosis and tubular atro...,\nEdited By: 07/08/16 - 1656 16192><MICROSCOPI...


In [2]:
data = pd.read_csv("data.csv")

In [3]:
data

,Unnamed: 0,Accession #,Gender,Age at Procedure,Raw Case Text,TCMR,ABMR,isRejection,poly_infection,IFTA,main_categories,other_categories,train_report
0,0,21SP-22339,Male,47,"FINAL DIAGNOSIS \n TRANSPLANT KIDNEY, SURVEILL...",0,0,0,0,mild,[' - Minimal nonspecific interstitial inflamma...,[' - Mild interstitial fibrosis and tubular at...,", MICROSCOPIC DESCRIPTION \n Light Microscopy ..."
1,1,21SP-22334,Male,38,"FINAL DIAGNOSIS \n TRANSPLANT KIDNEY, SURVEILL...",0,0,0,0,minimal,[' - No evidence of acute T-cell-mediated reje...,[' - Minimal interstitial fibrosis and tubular...,", MICROSCOPIC DESCRIPTION \n Light Microscopy ..."
2,2,21SP-22330,Female,43,"FINAL DIAGNOSIS \n TRANSPLANT KIDNEY, SURVEILL...",0,0,0,0,nosig,"['\\n TRANSPLANT KIDNEY, SURVEILLANCE BIOPSY (...",[' - No significant interstitial fibrosis or t...,", MICROSCOPIC DESCRIPTION \n Light Microscopy ..."
3,3,21SP-22328,Male,46,"FINAL DIAGNOSIS \n TRANSPLANT KIDNEY, BIOPSY (...",0,0,0,0,nosig,[' - Insufficient evidence for acute T-cell-me...,[' - No significant interstitial fibrosis or t...,", MICROSCOPIC DESCRIPTION \n Light Microscopy ..."
4,4,21SP-22116,Male,27,"FINAL DIAGNOSIS \n TRANSPLANT KIDNEY, TIME-ZER...",0,0,0,0,nosig,[],"[' - No significant glomerulosclerosis, inters...",", MICROSCOPIC DESCRIPTION \n Light Microscopy ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3424,3424,16:SP11594,F,39,<Ordered Procedures \n G&M Level 4\nIMMUNOHIST...,0,0,0,0,mild,[' No evidence of T-cell-mediated or antibody-...,[' Mild interstitial fibrosis and tubular atro...,\njw:ds \nEdited By: 07/11/16 - 1516 16192><MI...
3425,3425,16:SP11519,M,55,<Ordered Procedures \n G&M Level 4\nIMMUNOHIST...,0,0,0,0,noinfo,[' No evidence of T-cell-mediated or antibody-...,[],\nEdited By: 07/08/16 - 1601 16192><MICROSCOPI...
3426,3426,16:SP11518,M,61,<Ordered Procedures \n G&M Level 4\nIMMUNOHIST...,0,0,0,0,moderate,[' No evidence of polyomavirus infection'],[' Moderate (approaching severe) interstitial ...,\nEdited By: 07/08/16 - 1701 16192><MICROSCOPI...
3427,3427,16:SP11507,F,29,<Ordered Procedures \n G&M Level 4\nIMMUNOHIST...,0,0,0,0,mild,[' No evidence of antibody-mediated rejection'...,[' Mild interstitial fibrosis and tubular atro...,\nEdited By: 07/08/16 - 1656 16192><MICROSCOPI...


In [6]:
def have_target_rej(sent):
    target_lst = ["t-cell-mediated rejection", "antibody-mediated rejection"]
    for target_word in target_lst:
        if target_word in sent.lower():
            return True
    return False

def have_target_ifta(sent):
    target_lst = ["interstitial fibrosis","tubular atrophy"]
    for target_word in target_lst:
        if target_word in sent.lower():
            return True
    return False

def get_train_rej(report):
    sents = report.split(".")
    result = []
    flag = False
    for sent in sents:
        if "microscopic description" in sent.lower():
            result.append(sent)
            flag = True
            continue
#         elif "interpretation" in sent.lower():
#             break
        elif "banff scoring components" in sent.lower():
            break
            
        if flag and not have_target_rej(sent):
            result.append(sent)
            
    return ". ".join(result)

def get_train_ifta(report):
    sents = report.split(".")
    result = []
    flag = False
    for sent in sents:
        if "microscopic description" in sent.lower():
            result.append(sent)
            flag = True
            continue
#         elif "interpretation" in sent.lower():
#             break
        elif "banff scoring components" in sent.lower():
            break
            
        if flag and not have_target_ifta(sent):
            result.append(sent)
            
    return ". ".join(result)

In [7]:
data["train_rej"] = data["Raw Case Text"].apply(get_train_rej)

In [8]:
data["train_ifta"] = data["Raw Case Text"].apply(get_train_ifta)

In [15]:
data["train_rej"][2]

', MICROSCOPIC DESCRIPTION \\n Light Microscopy (LM):  The following LM findings are based on hematoxylin and eosin (H&E), periodic acid-Schiff (PAS), and Masson trichrome-stained sections.   The material submitted for LM contains two cores of almost all renal cortex.   More than 50 glomeruli are present for examination, of which 2 are globally sclerotic.   The non-sclerotic glomeruli appear immature and contain open capillary loops with no significant glomerulitis.   The capillary walls are of normal thickness and contours.   No significant mesangial matrix accumulation or mesangial hypercellularity is identified.   The tubulointerstitium is generally unremarkable with the tubules arranged in a back-to-back orientation.   No significant interstitial fibrosis, tubular atrophy, tubular injury, interstitial edema, interstitial inflammation, or tubulitis is present.   No viral cytopathic changes are observed.   The arteries and arterioles are unremarkable.   No endotheliitis is present.  

In [56]:
data.to_csv("data.csv")